In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import time
import requests

# Configuração da API OpenAI
KEY = 'YOUR_OPENAI_API_KEY'
URL = "https://api.openai.com/v1/engines/davinci-codex/completions"
HEADERS = {
    "Authorization": f"Bearer {KEY}",
    "Content-Type": "application/json"
}

# Leitura dos conjuntos de dados
arquivo_treinamento = '/content/drive/MyDrive/Colab Notebooks/2023/BRACIS_toxic/data/train_toldbr.csv'
arquivo_teste = '/content/drive/MyDrive/Colab Notebooks/2023/BRACIS_toxic/data/test_toldbr.csv'
dados_treinamento = pd.read_csv(arquivo_treinamento)
dados_teste = pd.read_csv(arquivo_teste)

# Seleção de exemplos few-shot
SEMENTE = 42
exemplos_toxicos = dados_treinamento[dados_treinamento['Toxic'] == 1.0].sample(5, random_state=SEMENTE)
exemplos_nao_toxicos = dados_treinamento[dados_treinamento['Toxic'] == 0.0].sample(5, random_state=SEMENTE)

# Criação do prompt com os exemplos
prompt_inicial = "Você está realizando análise de texto. O texto a seguir foi utilizado em um contexto de uma rede social. Responda apenas 'sim' ou 'não' caso o texto abaixo contenha discurso de ódio, ofensa, agressividade ou outras formas de toxicidade:\n"
for _, row in exemplos_toxicos.iterrows():
    prompt_inicial += f"Texto: {row['clean_text']} -> Resposta: sim\n"

for _, row in exemplos_nao_toxicos.iterrows():
    prompt_inicial += f"Texto: {row['clean_text']} -> Resposta: não\n"
prompt_inicial += "Agora classifique o texto a seguir: "

# Função para classificar texto com few-shot e Retry com Exponential Backoff
def classificar_texto(texto, max_retries=5):
    prompt_completo = prompt_inicial + texto
    data = {
        "prompt": prompt_completo,
        "max_tokens": 50,         # Ajuste conforme necessário
        "temperature": 0,
    }

    for i in range(max_retries):
        resposta = requests.post(URL, headers=HEADERS, json=data)
        if resposta.status_code == 200:
            return resposta.json()['choices'][0]['text'].strip()
        elif resposta.status_code == 429:  # Rate limited
            sleep_time = 2 ** (i + 1)  # Exponential backoff: 2, 4, 8, 16, ...
            print(f"Rate limited! Tentando novamente em {sleep_time} segundos...")
            time.sleep(sleep_time)
        else:
            print(f"Erro na requisição: {resposta.status_code} - {resposta.text}")
            sleep_time = 2 ** (i + 1)
            print(f"Esperando {sleep_time} segundos antes da próxima tentativa...")
            time.sleep(sleep_time)

    raise Exception("Número máximo de tentativas alcançado!")

# Classificar cada tweet usando few-shot
resultados = []
for tweet in dados_teste['clean_text']:
    resultado = classificar_texto(tweet)
    print(tweet)
    print(resultado + '\n')
    resultados.append(resultado)

# Salvar resultados
dados_teste['Classificação_ChatGPT'] = resultados
dados_teste.to_csv('/content/drive/MyDrive/Colab Notebooks/2023/BRACIS_toxic/data/resultados_classificacao_chatgpt_ToldBR_prompt_2_fewshot.csv', index=False)
print("Classificação concluída e resultados salvos.")
